In [1]:
import os, copy, sys
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
sys.path.extend(['../..', '../../../..'])
import trkd_roi_analysis as tra
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as mcolors
from matplotlib.colors import colorConverter
import matplotlib as mpl
import matplotlib.patches as mpatches

In [2]:
mouse_df_fnm = '../../../../mouse_df.csv'
mouse_df, _, _ = tra.set_up_mouse_info(mouse_df_fnm)

All mice:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13]
Mice with all of sessions 1-3 : [ 1  3  4  6  7  8  9 10 11 12 13]


<h2>Load data</h2>

In [3]:
# mask_fnm = '../../data/nway_union_tmasks.h5'
mask_fnm = '../../data/nway_union_tmasks__all_4_planes.h5'
nway_union_tmasks_df = pd.read_hdf(mask_fnm)

In [4]:
nway_union_tmasks_df

,mouse_n,layer,compartment,masks
0,6,L2/3,Dendrites,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,11,L5,Dendrites,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,3,L2/3,Somas,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3,4,L5,Somas,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


<h2>Main cell to plot data</h2>

In [6]:
%%time

mouse_ns = [11,4]
mouse_n_exceptions = []

# nwaydir = 'n-way-roi-matching'
compartments = {'soma':'Somas', 'dend':'Dendrites'}
sess_ns = np.array([1,2,3])

pdf_fnm = '../../plots/l5-matched-rois.pdf'

with PdfPages(pdf_fnm) as pdf:

    for mouse_n in mouse_ns:
        print('mouse ', mouse_n)
        mask = nway_union_tmasks_df['mouse_n']==mouse_n
        nway_union_tmasks = nway_union_tmasks_df[mask]['masks'].values[0]
        plane = mouse_df[mouse_df['mouse_n']==mouse_n]\
                    ['plane'].unique()[0]
        line = mouse_df[mouse_df['mouse_n']==mouse_n]\
                    ['line'].unique()[0]
        compartment = compartments[plane]
        layers = {'L23-Cux2':'L2/3', 'L5-Rbp4':'L5'}
        layer = layers[line]
        
        plt.figure(figsize=(50,50))
        plt.text(0.5, 0.5, '{} {}\n\nMouse {}'.format(layer, compartment, mouse_n),  
                 horizontalalignment='center', verticalalignment='center', size=100, style='italic')
        plt.xticks([],[])
        plt.yticks([],[])
        plt.axis('off')
        pdf.savefig()
        plt.close()

        
        fig, ax = plot_nway_union_figure(mouse_n, layer, compartment, nway_union_tmasks)
        pdf.savefig()
        plt.close()    

mouse  11
mouse  4
CPU times: user 7.88 s, sys: 927 ms, total: 8.8 s
Wall time: 8.17 s


<h2>Plotting function</h2>

In [5]:
def plot_nway_union_figure(mouse_n, layer, compartment, nway_union_tmasks):

    colors = [mcolors.CSS4_COLORS['dimgray'],
                  mcolors.CSS4_COLORS['orangered'],
                  mcolors.CSS4_COLORS['mediumblue']]
    levels=256
    alphas = 0.6*np.ceil(np.linspace(0, 0.8, levels+3))
    
    sess_natural_order = copy.deepcopy(sess_ns)
    sess_natural_order.sort()
    sess_nat_str = ', '.join(sess_natural_order.astype(str))

    fig, ax = plt.subplots(figsize=(50,50), constrained_layout=True)
    for k,_ in enumerate(sess_ns):
        cmap = mpl.colors.LinearSegmentedColormap.from_list(
            'my_cmap',['white', colors[k]],N=levels)
        cmap._init()
        cmap._lut[:,-1] = alphas
        im = plt.imshow(nway_union_tmasks[k,:], cmap=cmap)
    plt.xticks([], [])
    plt.yticks([], [])
    plt.title('N-way matches across permutations (Union - Conflicts)\n\n', fontsize=45, fontstyle='italic')
    patches = [ mpatches.Patch(color=colors[i], label="Session {l}".format(l=sess_ns[i]) ) 
               for i in range(len(sess_ns)) ]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, fontsize=40)

    return fig, ax